In [1]:
from pathlib import *
import numpy as np
import nibabel as nib
import mmimproc as ip

def append2fn(fn, newstr):
    return Path(fn).stem + newstr + ''.join(Path(fn).suffixes)

ssvolnum = 10     # integer volume number where steady state is acheived 

# set up file naming
datadir = ip.fs_local   # enter pathlib for BIDS root data directory
proj = 'toddandclark'   # enter project name
subj = 'badrestingstate'  # enter subject name (todo: subject list option for for loop)
niiname = 'rest_50125.nii'  # name of raw resting nifti file to process (todo: for loop if needed)
namedict  = {'datadir': datadir, 'proj': proj, 'subj': subj, 'niiname': niiname,
             'maskname': append2fn(niiname, '_mask'), 'snrmaskname': append2fn(niiname, '_snrmask')}

# load resting state file memory map
badvol = nib.load('{datadir}/{proj}/{subj}/{niiname}'.format(**namedict))
# make the numpy data array
badvoldata = np.asanyarray(badvol.dataobj).astype(np.float64)[...,ssvolnum:]
# resave with non steady state vols chopped off
namedict['chopvolfname'] = append2fn(niiname, '_'+str(badvoldata.shape[-1])+'vols')
chopvolfname = '{datadir}/{proj}/{subj}/{chopvolfname}'.format(**namedict)
nib.save(nib.Nifti1Image(badvoldata, badvol.affine), chopvolfname)

# make 3 part mask
bvsh = badvoldata.shape
badvolmask = np.zeros(bvsh).astype(np.int)
badvolmask[39:41 ,25:31 ,17:20, :] = 1  # Left operc
badvolmask[25:27 ,25:31 ,17:20, :] = 2  # Right operc
# define left anterior superior corner voxels for noise estimation
oobcoord = np.index_exp[bvsh[0] - 5:bvsh[0] - 1, bvsh[1] - 10:bvsh[1] - 1, bvsh[2] - 4:bvsh[2] - 1, :]
badvolmask[oobcoord] = 3 # out of body area left anterior superior corner
# save mask to nii file
maskfname = '{datadir}/{proj}/{subj}/{maskname}'.format(**namedict)
nib.save(nib.Nifti1Image(badvolmask, badvol.affine), maskfname)

In [2]:
# generate snr data and nii vol with stats embedded
mbadvoldata_roi1 = np.where(badvolmask == 1, badvoldata, np.nan)
#mbadvoldata_roi1[mbadvoldata_roi1 == 0] = np.nan
mean_roi1 = np.nanmean(mbadvoldata_roi1, axis=(0, 1, 2))
mbadvoldata_roi2 = np.where(badvolmask == 2, badvoldata, np.nan)
#mbadvoldata_roi2[mbadvoldata_roi2 == 0] = np.nan
mean_roi2 = np.nanmean(mbadvoldata_roi2, axis=(0, 1, 2))
mbadvoldata_roi3 = np.where(badvolmask == 3, badvoldata, np.nan)
#mbadvoldata_roi3[mbadvoldata_roi3 == 0] = np.nan
std_roi3 = np.nanstd(mbadvoldata_roi3, axis=(0, 1, 2))
snr1 = mean_roi1 / std_roi3
snr2 = mean_roi2 / std_roi3
snrmask = np.zeros(bvsh).astype(np.float64)
snrmask[39:41 ,25:31 ,17:20, :] = snr1  # Left operc snr
snrmask[25:27 ,25:31 ,17:20, :] = snr2  # Right operc snr
snrmask[oobcoord] = std_roi3            # out of body sigma left anterior superior corner
snrmaskfname = '{datadir}/{proj}/{subj}/{snrmaskname}'.format(**namedict)
nib.save(nib.Nifti1Image(snrmask, badvol.affine), snrmaskfname)

In [9]:
mean_roi1 / np.nanstd(mbadvoldata_roi1, axis=(0, 1, 2))

array([36.06815891, 32.10392065, 32.97992554, 31.90238644, 36.44659696,
       34.87586409, 34.38011401, 36.90432154, 32.48211348, 39.20472128,
       29.39248716, 28.51879816, 30.36296408, 32.4241023 , 32.7546494 ,
       30.98272428, 32.12358084, 33.09224183, 30.92377563, 34.29137034,
       36.05095186, 36.02038522, 33.23324626, 38.94847827, 40.44340274,
       28.92314169, 30.12318281, 34.73972326, 35.55866207, 36.73316864,
       36.80839758, 34.96263159, 35.32341248, 36.48809607, 32.05295556,
       32.15612687, 35.01589355, 35.40787506, 30.38443877, 34.52782536,
       30.58400202, 31.31574145, 32.55471537, 35.39436422, 35.51816885,
       35.24520074, 35.13332173, 33.71853217, 34.56526055, 31.69803305,
       36.37228429, 33.83839549, 30.16721424, 32.60634236, 33.43680389,
       33.09458039, 34.43454384, 29.42352561, 36.22524756, 32.86633096,
       37.04229269, 36.05754336, 32.82894566, 34.61921797, 34.6223311 ,
       33.78764281, 33.76118166, 39.38180437, 38.55695035, 34.62

In [10]:
snr1

array([142.78218662, 135.29327348, 146.28492757, 139.59648247,
       159.06517073, 131.31192463, 123.31710537, 146.06302743,
       118.71496564, 137.92075266, 131.016042  , 144.6714073 ,
       130.24624342, 137.99871292, 131.80710253, 141.02339568,
       129.83731791, 124.06243592, 141.4423464 , 137.40524146,
       134.21602334, 125.22367274, 137.26989912, 141.38660924,
       118.92585186, 146.23554008, 147.2412074 , 136.12058054,
       152.98838029, 146.44963237, 142.22548683, 139.39740557,
       131.31812776, 149.06792103, 144.70053856, 137.81523357,
       131.93783961, 144.34961927, 146.93331642, 130.19618178,
       152.59803338, 118.50556082, 125.59902144, 135.15428779,
       144.60342905, 124.46103393, 129.53476101, 130.11328635,
       128.97377218, 136.88025511, 138.91210999, 126.16063866,
       120.99478512, 112.73825712, 144.72741854, 149.37907193,
       132.85585666, 152.42177021, 121.32062708, 145.59991894,
       136.71266796, 139.92596314, 121.40935805, 127.74